In [2]:
import numpy as np
import scipy 

# Gaussian mixture

$\pi (x) = \frac{1}{2(2\pi)^{d/2}} \left( e ^{\frac{-| x-a|^2}{2}}  + e ^{\frac{-| x+a|^2}{2}} \right), \quad x \in\mathbb R^d$

$U(x) = \frac{1}{2} \|x - a\|_2^2 - \text{log}(1 + e^{-2x^\top a})$

$\nabla U(x) = x-a +\frac{2a}{(1 + e^{2 x^\top a})}$

$ m = 1 - \|a \|_2^2 \quad $ (strongly convex function)

$M = 1 \quad$  (Lipschitz continuous gradient)

$a = (\frac{1}{\sqrt{2d}}, \dots, \frac{1}{\sqrt{2d}})$

### d = 2, h = 0.1

In [3]:
d = 2
h = 0.1

In [4]:
a = np.ones((d,1)) / np.sqrt(2*d)

def f_grad(x):
    return x-a+2*a/(1 + np.exp(2* (x.T @ a)))

def ULA_gm(a,d,N,h, K = 10000):
    vkh = np.empty((K + N,d,1))
    ksi = np.random.randn(K+N,d,1)
    vkh[0] = (np.random.normal(0,1,d)).reshape(d,1)
    for i in range(1,K+N):
        grad = f_grad(vkh[i-1])
        vkh[i] = vkh[i-1] - h*grad + np.sqrt(2*h) * ksi[i]
    return vkh[K:], ksi[K:]

In [5]:
np.random.seed(2342)
X, Z = ULA_gm(a,d, 100000, h)

In [6]:
def local_weighted_estimator(X):
    return X.sum(axis = 1).mean()

In [7]:
print ("Weighted estimator = ",local_weighted_estimator(X))

Weighted estimator =  -0.026755673056072866


In [13]:
def ULA_from_initial(x_initial,a,d,N, h):
    vkh = np.empty((N,d,1))
    vkh_grad = np.empty((N,d,1))
    ksi = np.random.randn(N,d,1)
    vkh[0] = x_initial
    for i in range(1,N):
        grad = f_grad(vkh[i-1])
        vkh_grad[i-1] = grad
        vkh[i] = vkh[i-1] - h*grad + np.sqrt(2*h) * ksi[i]
    vkh_grad[-1] = f_grad(vkh[-1])
    return vkh, ksi, vkh_grad

def generate_paths(x_initial,N_train,a,d,N, h):
    XX = []
    ZZ = []
    GG = []
    for i in range(N_train):
        X, Z, G = ULA_from_initial(x_initial[-i],a,d,N, h)
        XX.append(X)
        ZZ.append(Z)
        GG.append(G)
    return np.array(XX),np.array(ZZ), np.array(GG)

In [14]:
np.random.seed(123)
XX, ZZ, _ = generate_paths(X,1000,a,d,1000, h)